In [ ]:
%pylab
from scipy.stats import sem
import sfg2d.talk
import json

%load_ext autoreload
%autoreload 2
import sfg2d

Import Data

In [ ]:
record = sfg2d.SfgRecord('')
record.name = ''
record.lname = ''
record.pump_freq = 0
record

# Static Spectrum

In [ ]:
sfg2d.plot_record_static(record, save=True)

# Contour Plot

In [ ]:
sfg2d.plot_record_contour(record, save=True, levels=(84, 110, 15))

# Four Level Fit

In [ ]:
sl = slice(2550, 2650) # Slice in Wavenumbers

x, y, yerr = record.trace(roi_wavenumber=sl)
x = x.squeeze()
y = y.squeeze()
yerr = yerr.squeeze()
fargs = {
     'limit_c': (-10, 10),
     'limit_mu': (-800, 0),
     'limit_s': (0.0001, 1),
     'limit_t1': (1, 10000),
     'limit_t2': (1, 10000),
     'fix_s': False,
     'fix_t1': False,
     'fix_t2': True,
     'fix_c': False,
     'fix_mu': False,
     's': 0.044,
     't1': 556.0,
     't2': 700.0,
     'c': 1.22,
     'mu': -123,

}
name = '{}-{}'.format(sl.start, sl.stop)
record.models[name] = sfg2d.models.FourLevelMolKinM(
    xdata=x, 
    ydata=y.squeeze(), 
    sigma=yerr.squeeze(),
    fitarg=fargs,
    rtol=2.09e-9,
    atol=3.00e-9,
    gSigma=264,
)
model = record.models[name]
#model.box_coords = (1600, 0.9)
sfg2d.plot_model_trace(
    record.name  + "_" + name, record.models[name], 
    record, ylim=None, xlim=None, save=False,
    title='{} {}-{} 1/cm'.format(record.lname, sl.start, sl.stop)
)

record.models

In [ ]:
model.minuit.migrad()  # Run the minimization
sfg2d.plot_model_trace(
    record.name  + "_" + name, record.models[name], record, 
    ylim=None, xlim=None, save=False,
    title='{} {}-{} 1/cm'.format(record.lname, sl.start, sl.stop)
)
fit_results[record.name] = sfg2d.core.get_fit_results(record)

## Run the Fits

In [ ]:
for name, model in record.models.items():
    print(name)
    stats, params = model.minuit.migrad()
    with open("cache/{}_trace_pump{}_{}.json".format(record.name, name, record.pump_freq), "w") as file:
        json.dump(model.minuit.fitarg, file, indent=2)

## Plot Fit Results

In [ ]:
#record.models['2550-2650'].box_coords = (1600, .900)

for name, model in record.models.items():
    sfg2d.plot_model_trace(name, model, record, ylim=(.87, 1.08), xlim=(-1100, 5000), save=True)

# CorssCorrelation Fit

In [ ]:
cc = sfg2d.SfgRecord('01_cc_gold_pr1_pu1_gal1_ppp.dat')
x, y, yerr = cc.trace(prop='rawData', prop_kwgs={}, roi_spectra=[2])
gm = sfg2d.models.GaussianModelM(
    x, y.squeeze(), sigma=yerr.squeeze(),
    fitarg={
        'A': 500000,
        'mu': -200,
        'sigma': 200,
        'c':300
    },
    #roi=slice(3, -5)
)
gm.box_coords = (-500, 400)
gm.minuit.migrad()
sfg2d.plot_model_trace(
    'CorssCorrelation', 
    gm, 
    cc, 
    xlim=None, 
    ylim=None, 
    save=True,
    fname_format='figures/cc.pdf'
)